необходимо добавить датасет https://kaggle.com/datasets/92ee97832586af60fef4ac42dce5e62d428d7c561eb10a8e7746fb4d9f9d175b

In [10]:
# !pip install pudub
# !pip install librosa
!pip install git+https://github.com/huggingface/datasets

In [8]:
import os
import pandas as pd
import random
import shutil
from scipy.io.wavfile import write
from pydub import AudioSegment
import librosa
import torch
import torchaudio

In [14]:
# какая-то страшная структура, пришлось метаданные искать вот так
for root, dirs, files in os.walk("/kaggle/input/cv-corpus-2023-03-09-ru/"):
    for filename in files:
        if not filename.endswith(".mp3"):
            print(root, filename)

/kaggle/input/cv-corpus-2023-03-09-ru/ __notebook_source__.ipynb
/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpus-13.0-2023-03-09-ru/14407546371 14403167155/cv-corpus-13.0-2023-03-09/ru test.tsv
/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpus-13.0-2023-03-09-ru/14407546371 14403167155/cv-corpus-13.0-2023-03-09/ru dev.tsv
/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpus-13.0-2023-03-09-ru/14407546371 14403167155/cv-corpus-13.0-2023-03-09/ru train.tsv
/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpus-13.0-2023-03-09-ru/14407546371 14403167155/cv-corpus-13.0-2023-03-09/ru validated.tsv
/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpus-13.0-2023-03-09-ru/14407546371 14403167154/cv-corpus-13.0-2023-03-09/ru invalidated.tsv
/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpus-13.0-2023-03-09-ru/14407546371 14403167154/cv-corpus-13.0-2023-03-09/ru other.tsv
/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpus-13.0-2023-03-09-ru/14407546371 14407546351/cv-corpus-13.0-2023-03-09/ru reported.tsv


In [9]:
df_val = pd.read_csv('/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpus-13.0-2023-03-09-ru/14407546371 14403167155/cv-corpus-13.0-2023-03-09/ru/validated.tsv', sep='\t')
df_val

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,01648c1613193e9f272373de6867c358d542ae833de4da...,common_voice_ru_26587318.mp3,"К сожалению, эти предложения не нашли отражени...",2,0,NaN,NaN,NaN,NaN,ru,NaN
1,0371548939227335ee65bbe0e40f05bc65c75d1e6f4cb5...,common_voice_ru_36292648.mp3,"Толпа озвереет, будет тереться, ощетинит ножки...",2,0,NaN,NaN,NaN,NaN,ru,NaN
2,067e02093c9ab72ec46d773147c822cb1f839efd29fa65...,common_voice_ru_36461021.mp3,А жизнь ее была не весела.,2,0,NaN,NaN,NaN,NaN,ru,NaN
3,081340f4cbaab50d8b9606a52488eb0614edd77eb782a4...,common_voice_ru_19468869.mp3,"Если не будет возражений, я буду считать, что ...",2,0,NaN,NaN,NaN,NaN,ru,NaN
4,08ce6bb4ac95d3cb2e5b0f1a54ac93d065d41caede4889...,common_voice_ru_31928067.mp3,Да ты зачем собственно приехал?,2,0,NaN,NaN,NaN,NaN,ru,NaN
...,...,...,...,...,...,...,...,...,...,...,...
148637,3742174ae31b78b11558814a7196a5eec0f489ad3e728a...,common_voice_ru_30153905.mp3,Неделя,2,0,twenties,male,NaN,NaN,ru,NaN
148638,3742174ae31b78b11558814a7196a5eec0f489ad3e728a...,common_voice_ru_30153906.mp3,Запирать окна и двери!,2,0,twenties,male,NaN,NaN,ru,NaN
148639,3742174ae31b78b11558814a7196a5eec0f489ad3e728a...,common_voice_ru_30153907.mp3,Первоначальные сведения по оккультизму.,2,1,twenties,male,NaN,NaN,ru,NaN
148640,3742174ae31b78b11558814a7196a5eec0f489ad3e728a...,common_voice_ru_30153908.mp3,"До дверей магазина мы шли вместе, но в магазин...",3,0,twenties,male,NaN,NaN,ru,NaN


In [10]:
validated_data = set(df_val['path'].values)

In [11]:
file_names=[]
paths = []
full_paths=[]

TRAIN_TEST_SPLIT = 0.02
for root, dirs, files in os.walk("/kaggle/input/cv-corpus-2023-03-09-ru/"):
    for filename in files:
        if filename.endswith(".mp3"):
            if  filename in validated_data:
                if random.random()>TRAIN_TEST_SPLIT:
                    file_names.append(f"train/{filename.split('.')[0]}.wav")
                else:
                    file_names.append(f"test/{filename.split('.')[0]}.wav")
                paths.append(filename)
                full_paths.append(os.path.join(root, filename))

    df = pd.DataFrame(columns=['file_name', 'full_path', 'path'])
df['file_name'] = file_names
df['full_path'] = full_paths
df['path'] = paths
df

,file_name,full_path,path
0,train/common_voice_ru_26682556.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,common_voice_ru_26682556.mp3
1,train/common_voice_ru_26682578.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,common_voice_ru_26682578.mp3
2,train/common_voice_ru_26682580.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,common_voice_ru_26682580.mp3
3,train/common_voice_ru_26682559.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,common_voice_ru_26682559.mp3
4,train/common_voice_ru_26682558.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,common_voice_ru_26682558.mp3
...,...,...,...
148637,train/common_voice_ru_21906546.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,common_voice_ru_21906546.mp3
148638,train/common_voice_ru_21906558.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,common_voice_ru_21906558.mp3
148639,train/common_voice_ru_21906485.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,common_voice_ru_21906485.mp3
148640,train/common_voice_ru_21906556.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,common_voice_ru_21906556.mp3


In [12]:
df = df.merge(df_val, 'inner', on='path')
df['answer'] = df['sentence']
df['source'] = 'common_voice'
df['duration'] = None
df = df[['file_name', 'full_path', 'answer', 'duration', 'source']]
df

,file_name,full_path,answer,duration,source
0,train/common_voice_ru_26682556.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,"Кстати, когда вы передадите им ключи от Замка?",None,common_voice
1,train/common_voice_ru_26682578.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Слово имеет представитель Австрии.,None,common_voice
2,train/common_voice_ru_26682580.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Для придания нового импульса многосторонним ус...,None,common_voice
3,train/common_voice_ru_26682559.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,"Индия твердо выступает за глобальное, недискри...",None,common_voice
4,train/common_voice_ru_26682558.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Он вовсе не был зануден.,None,common_voice
...,...,...,...,...,...
148637,train/common_voice_ru_21906546.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Firefox,None,common_voice
148638,train/common_voice_ru_21906558.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Мы будем содействовать активизации работы наци...,None,common_voice
148639,train/common_voice_ru_21906485.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,один,None,common_voice
148640,train/common_voice_ru_21906556.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,нет,None,common_voice


In [13]:
df_train = df[df['file_name'].apply(lambda x: x.startswith('train'))]
df_train.reset_index(drop=True, inplace=True)
df_train

,file_name,full_path,answer,duration,source
0,train/common_voice_ru_26682556.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,"Кстати, когда вы передадите им ключи от Замка?",None,common_voice
1,train/common_voice_ru_26682578.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Слово имеет представитель Австрии.,None,common_voice
2,train/common_voice_ru_26682580.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Для придания нового импульса многосторонним ус...,None,common_voice
3,train/common_voice_ru_26682559.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,"Индия твердо выступает за глобальное, недискри...",None,common_voice
4,train/common_voice_ru_26682558.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Он вовсе не был зануден.,None,common_voice
...,...,...,...,...,...
145601,train/common_voice_ru_21906546.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Firefox,None,common_voice
145602,train/common_voice_ru_21906558.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Мы будем содействовать активизации работы наци...,None,common_voice
145603,train/common_voice_ru_21906485.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,один,None,common_voice
145604,train/common_voice_ru_21906556.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,нет,None,common_voice


In [14]:
df_test = df[df['file_name'].apply(lambda x: x.startswith('test'))]
df_test.reset_index(drop=True, inplace=True)
df_test

,file_name,full_path,answer,duration,source
0,test/common_voice_ru_26958833.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Пока не удается надежно вернуть мировую эконом...,None,common_voice
1,test/common_voice_ru_24359708.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Словарь,None,common_voice
2,test/common_voice_ru_18968873.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Это перекликается со словами президента Сальвы...,None,common_voice
3,test/common_voice_ru_18968909.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Сорт «золотой налив».,None,common_voice
4,test/common_voice_ru_27022302.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,"Однако вскоре они чувствуют взаимные уколы, за...",None,common_voice
...,...,...,...,...,...
3031,test/common_voice_ru_19340547.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Объектом их рассмотрения были четыре фундамент...,None,common_voice
3032,test/common_voice_ru_26072910.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,"Нет, ты можешь велеть самовар принять.",None,common_voice
3033,test/common_voice_ru_28781651.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,Я рада.,None,common_voice
3034,test/common_voice_ru_20365262.wav,/kaggle/input/cv-corpus-2023-03-09-ru/cv-corpu...,"Откройте дверь, пожалуйста.",None,common_voice


In [ ]:
# целиком на кэгле не помещается датасет, пришлось разбить трейн на две части
df_part1 = df_train[:72535]
df_part2 = df_train[72535:]

In [16]:
os.mkdir("cv-corpus-2023-03-09-ru_modified")
os.mkdir("cv-corpus-2023-03-09-ru_modified/train")

In [17]:
lens = []
for index, row in df_part1.iterrows():
    if index%2500==0:
        print(index)
    sound = AudioSegment.from_mp3(row['full_path'])
    sound = sound.set_frame_rate(16000)
    sound = sound.set_channels(1)
    lens.append(len(sound)/1000)
    sound.export(f"/kaggle/working/cv-corpus-2023-03-09-ru_modified/{row['file_name']}", format='wav')

0
2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500


In [18]:
df_part1['duration']=lens
df_part1.drop(columns='full_path', inplace=True)
df_part1

/tmp/ipykernel_32/4241519716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_part1['duration']=lens
/tmp/ipykernel_32/4241519716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_part1.drop(columns='full_path', inplace=True)


,file_name,answer,duration,source
0,train/common_voice_ru_26682556.wav,"Кстати, когда вы передадите им ключи от Замка?",4.140,common_voice
1,train/common_voice_ru_26682578.wav,Слово имеет представитель Австрии.,2.988,common_voice
2,train/common_voice_ru_26682580.wav,Для придания нового импульса многосторонним ус...,7.668,common_voice
3,train/common_voice_ru_26682559.wav,"Индия твердо выступает за глобальное, недискри...",6.516,common_voice
4,train/common_voice_ru_26682558.wav,Он вовсе не был зануден.,2.700,common_voice
...,...,...,...,...
72530,train/common_voice_ru_30578071.wav,Господи Боже мой!,2.628,common_voice
72531,train/common_voice_ru_26961354.wav,Именно поэтому Словения выдвинула свою кандида...,5.760,common_voice
72532,train/common_voice_ru_26961355.wav,"И тотчас же привези ответ, — сказала она посла...",4.896,common_voice
72533,train/common_voice_ru_26961346.wav,"Беженцы, которых мы призываем вернуться, будут...",7.668,common_voice


In [19]:
print(sum(lens)/3600)

102.69883805555705


In [20]:
df_part1.to_csv('/kaggle/working/cv-corpus-2023-03-09-ru_modified/metadata.csv')

In [21]:
# создаем датасет локально
from datasets import load_dataset

dataset = load_dataset("audiofolder", data_dir='/kaggle/working/cv-corpus-2023-03-09-ru_modified')

Resolving data files:   0%|          | 0/72535 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/default-17d0dd8e106ed7cc/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# загружаем на HuggingFace

from huggingface_hub import login
login(token='') # добавить write токен нужного аккаунта HuggingFace 

DATASET_NAME = 'aanosov/common_voice_ru_part1'

dataset.push_to_hub(DATASET_NAME, private=True)

In [23]:
# удаляем локально файлы первой части, повторяем операцию со второй частью трейна
shutil.rmtree("cv-corpus-2023-03-09-ru_modified")

In [24]:
os.mkdir("cv-corpus-2023-03-09-ru_modified")
os.mkdir("cv-corpus-2023-03-09-ru_modified/train")

In [25]:
lens = []
for index, row in df_part2.iterrows():
    if index%2500==0:
        print(index)
    sound = AudioSegment.from_mp3(row['full_path'])
    sound = sound.set_frame_rate(16000)
    sound = sound.set_channels(1)
    lens.append(len(sound)/1000)
    sound.export(f"/kaggle/working/cv-corpus-2023-03-09-ru_modified/{row['file_name']}", format='wav')

75000
77500
80000
82500
85000
87500
90000
92500
95000
97500
100000
102500
105000
107500
110000
112500
115000
117500
120000
122500
125000
127500
130000
132500
135000
137500
140000
142500
145000


In [26]:
df_part2['duration']=lens
df_part2.drop(columns='full_path', inplace=True)
df_part2

/tmp/ipykernel_32/3118594297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_part2['duration']=lens
/tmp/ipykernel_32/3118594297.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_part2.drop(columns='full_path', inplace=True)


,file_name,answer,duration,source
72535,train/common_voice_ru_32358215.wav,Перемены необходимы для выживания человечества.,4.320,common_voice
72536,train/common_voice_ru_32358155.wav,Мы должны оживить застопорившийся реформаторск...,7.560,common_voice
72537,train/common_voice_ru_32358130.wav,"Она прежде всего женщина с сердцем, ты вот уви...",6.300,common_voice
72538,train/common_voice_ru_32358140.wav,Я даже сама выдумала одну мастику.,4.680,common_voice
72539,train/common_voice_ru_32358231.wav,"Нет, это действительно я.",4.140,common_voice
...,...,...,...,...
145601,train/common_voice_ru_21906546.wav,Firefox,2.136,common_voice
145602,train/common_voice_ru_21906558.wav,Мы будем содействовать активизации работы наци...,6.816,common_voice
145603,train/common_voice_ru_21906485.wav,один,1.944,common_voice
145604,train/common_voice_ru_21906556.wav,нет,2.016,common_voice


In [27]:
print(sum(lens)/3600)

104.0498775000031


In [28]:
df_part2.to_csv('/kaggle/working/cv-corpus-2023-03-09-ru_modified/metadata.csv')

In [ ]:
# создаем датасет
from datasets import load_dataset

dataset = load_dataset("audiofolder", data_dir='/kaggle/working/cv-corpus-2023-03-09-ru_modified')

In [30]:
# загружаем на HuggingFace

DATASET_NAME = 'aanosov/common_voice_ru_part2'

dataset.push_to_hub(DATASET_NAME, private=True)

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/24 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

...

Creating parquet from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/3044 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
# повторяем все с тестом
shutil.rmtree("cv-corpus-2023-03-09-ru_modified")

In [32]:
os.mkdir("cv-corpus-2023-03-09-ru_modified")
os.mkdir("cv-corpus-2023-03-09-ru_modified/test")

In [33]:
lens = []
for index, row in df_test.iterrows():
    if index%2500==0:
        print(index)
    sound = AudioSegment.from_mp3(row['full_path'])
    sound = sound.set_frame_rate(16000)
    sound = sound.set_channels(1)
    lens.append(len(sound)/1000)
    sound.export(f"/kaggle/working/cv-corpus-2023-03-09-ru_modified/{row['file_name']}", format='wav')

0
2500


In [34]:
df_test['duration']=lens
df_test.drop(columns='full_path', inplace=True)
df_test

/tmp/ipykernel_32/185642375.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['duration']=lens
/tmp/ipykernel_32/185642375.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(columns='full_path', inplace=True)


,file_name,answer,duration,source
0,test/common_voice_ru_26958833.wav,Пока не удается надежно вернуть мировую эконом...,6.048,common_voice
1,test/common_voice_ru_24359708.wav,Словарь,2.160,common_voice
2,test/common_voice_ru_18968873.wav,Это перекликается со словами президента Сальвы...,6.984,common_voice
3,test/common_voice_ru_18968909.wav,Сорт «золотой налив».,2.736,common_voice
4,test/common_voice_ru_27022302.wav,"Однако вскоре они чувствуют взаимные уколы, за...",6.480,common_voice
...,...,...,...,...
3031,test/common_voice_ru_19340547.wav,Объектом их рассмотрения были четыре фундамент...,6.984,common_voice
3032,test/common_voice_ru_26072910.wav,"Нет, ты можешь велеть самовар принять.",3.384,common_voice
3033,test/common_voice_ru_28781651.wav,Я рада.,2.376,common_voice
3034,test/common_voice_ru_20365262.wav,"Откройте дверь, пожалуйста.",2.520,common_voice


In [35]:
print(sum(lens)/3600)

4.340889166666654


In [36]:
df_test.to_csv('/kaggle/working/cv-corpus-2023-03-09-ru_modified/metadata.csv')

In [37]:
# создаем датасет
from datasets import load_dataset

dataset = load_dataset("audiofolder", data_dir='/kaggle/working/cv-corpus-2023-03-09-ru_modified')

Resolving data files:   0%|          | 0/3036 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/default-0c0c23263a58e465/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [38]:
# загружаем на HuggingFace

DATASET_NAME = 'aanosov/common_voice_ru_test'

dataset.push_to_hub(DATASET_NAME, private=True)

Map:   0%|          | 0/1518 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1518 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]